In [1]:
# Load libraries
#!pip install -r /work/NLP_IMDb_Exam/requirements.txt
import numpy as np
import pandas as pd
import torch
import datasets
import evaluate
import seaborn as sns
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

2024-12-30 18:35:49.234616: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-30 18:35:49.245218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735580149.256905    4615 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735580149.260466    4615 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-30 18:35:49.273925: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model_dict= {
    1 :{
        "name" : "MiniLM_L6",
        "huggingface" : "sentence-transformers/all-MiniLM-L6-v2",
        },

    2 :{
        "name" : "MPNET_base",
        "huggingface" : 'sentence-transformers/all-mpnet-base-v2',
        },    

    3 :{
        "name" : "Instructor",
        "huggingface" : "hkunlp/instructor-large",
    },}

rating_type={
    1 : "average_rating",
    2 : "rating",
}


# Choose a model for a pseudo-function
Chosen_Model= 2
Rating_to_Drop = 1

In [3]:
# Read embeddings from CSV 
#Base 
data_path= f'../Data/{model_dict[Chosen_Model]["name"]}/{model_dict[Chosen_Model]["name"]}.csv'
Embeddings= pd.read_csv(data_path)
Reviews= Embeddings['review']
Embeddings= Embeddings.drop(columns=[rating_type[Rating_to_Drop], 'review'])
model_dict[Chosen_Model]['Embeddings'] = Embeddings
model_dict[Chosen_Model]['Embeddings']

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,rating
0,-0.000947,0.012226,0.007005,0.055534,-0.014657,0.002093,-0.074283,-0.039043,-0.032846,-0.052571,...,-0.017689,-0.084283,0.017695,0.004549,0.043206,-0.031605,0.011563,0.031745,0.002231,8
1,-0.052814,0.075769,-0.001993,0.000877,-0.011637,0.033047,-0.067636,0.008955,-0.012041,0.013748,...,-0.019201,0.009876,0.021684,-0.033824,-0.027393,-0.003892,0.050564,-0.012418,0.008505,9
2,0.009295,0.026960,-0.008366,0.004835,-0.004934,0.028715,-0.028778,-0.024467,-0.058873,0.007024,...,0.013301,-0.079699,0.032424,-0.007187,0.063945,-0.006870,0.012035,0.040202,-0.007758,3
3,0.013212,-0.013945,0.020238,-0.017202,0.064402,-0.022100,0.005692,-0.038482,-0.045942,-0.008018,...,0.060098,-0.055245,-0.005580,0.001606,-0.046693,0.006035,0.012392,0.028939,0.009629,9
4,0.017525,0.003245,-0.025342,0.028730,-0.047887,0.027611,0.001204,0.022175,-0.012371,-0.023801,...,-0.013046,-0.054086,0.037302,0.000146,-0.001943,-0.079416,-0.028491,0.051881,-0.015033,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.010382,0.053012,-0.032119,0.057019,0.025475,-0.019095,0.019149,0.024946,0.026985,0.030446,...,0.050177,-0.005847,-0.019756,-0.009247,-0.033641,0.019529,-0.020493,-0.026690,0.000773,1
99996,-0.015336,-0.002935,-0.000669,-0.000911,-0.042267,0.010768,-0.054165,0.000786,-0.031114,-0.030957,...,0.043812,-0.039896,-0.019493,-0.024774,-0.043664,-0.004756,0.036723,0.051980,0.003448,10
99997,-0.012683,0.065704,-0.005387,0.006981,-0.057868,0.001411,-0.013149,0.002003,-0.113928,0.008597,...,0.042010,-0.067297,-0.011826,-0.004276,-0.066133,0.005304,0.036001,0.012250,-0.009513,10
99998,-0.036789,-0.032421,0.001179,0.038486,-0.001766,0.029219,-0.119113,-0.010546,0.012335,0.051544,...,-0.024835,0.073117,0.004253,-0.016748,-0.011802,0.027796,0.040786,-0.010568,0.002672,10


In [4]:
def positive_to_negative_vector(Positive, Negative):
    """
    Takes a positive and an negative data point and defines the vector spanning both vectors.
    """
    posneg_vector = Positive.mean().to_frame().T-Negative.mean().to_frame().T
    posneg_vector = pd.DataFrame(posneg_vector)
    return posneg_vector
    




In [5]:
Old_Texts = np.array([
    "That is ancient!",
    "This house was build in 1933",
    "The past",
    "Before the beginning"
])

New_Texts = np.array([
    "This is very new",
    "Very modern architechture is used for building this house",
    "The present is now",
    "The future",
    "In the year of 2024"
])

In [ ]:
Old_Texts = np.array([
    "This film redefines the action genre, delivering heart-pounding sequences and jaw-dropping stunts. A must-watch for adrenaline junkies!",
    "A deeply moving tale that tugs at the heartstrings and leaves you with a renewed sense of hope. Truly unforgettable.",
    "The visual effects are nothing short of breathtaking. Every frame is a work of art that immerses you completely.",
    "The cast's chemistry and brilliant performances bring the story to life in the most authentic and engaging way.",
    "A cinematic masterpiece with impeccable direction that seamlessly blends drama, suspense, and emotion.",
    "The characters are so well-developed and relatable that you can't help but get invested in their journey.",
    "An absolute laugh riot from start to finish! The witty dialogue and hilarious antics are sure to leave you in stitches.",
    "The world-building in this movie is unparalleled. Every detail is carefully crafted, creating a universe you'll never want to leave.",
    "A powerful and inspiring story that leaves you motivated to chase your dreams and overcome any obstacles.",
    "The music complements the story beautifully, elevating emotional moments and adding depth to every scene.",
    "This movie keeps you on the edge of your seat with its clever twists and turns. A gripping ride you won't forget.",
    "A refreshing take on a familiar theme, offering a perspective that feels both innovative and deeply resonant.",
    "Perfect for audiences of all ages, this movie delivers laughter, lessons, and love in equal measure.",
    "The cinematography is a visual feast, capturing both the grandeur of the setting and the intimacy of the characters' emotions.",
    "A delightful story that warms your heart and reminds you of the simple joys in life.",
    "An exhilarating journey filled with excitement, danger, and triumph. An epic adventure for the ages.",
    "The actors' raw and genuine performances make you forget you're watching a movie. Pure artistry.",
    "This film masterfully combines humor, drama, and action, making it a rollercoaster of emotions from start to finish.",
    "A vibrant celebration of culture and tradition, beautifully portrayed with authenticity and reverence.",
    "A film that transcends time with its universal themes and captivating storytelling. Destined to become a classic."
])

New_Texts = np.array([
    "This movie lacks any sense of direction, leaving the audience confused and frustrated.",
    "The storyline is painfully predictable, offering nothing new or exciting.",
    "Poorly written characters make it impossible to care about what happens to them.",
    "The acting is wooden and emotionless, making every scene feel forced and lifeless.",
    "A complete waste of stunning visuals due to a hollow and uninspired plot.",
    "The humor feels forced and falls flat, making the comedy aspect unbearable.",
    "Pacing issues plague the movie, with some parts dragging endlessly while others feel rushed.",
    "The dialogue is cringeworthy and unnatural, detracting from the overall experience.",
    "A disappointing sequel that fails to capture the magic of the original.",
    "The special effects are overused, overshadowing the weak storytelling.",
    "This movie tries too hard to be edgy but ends up being obnoxious and shallow.",
    "The soundtrack is forgettable and adds no value to the film.",
    "An overstuffed plot with too many subplots that go nowhere.",
    "The lack of chemistry between the leads makes their relationship unconvincing.",
    "The ending is abrupt and unsatisfying, leaving more questions than answers.",
    "An unoriginal rehash of better films, lacking any creativity or fresh ideas.",
    "The movie’s tone is inconsistent, making it hard to take seriously.",
    "Unnecessarily long runtime with scenes that add nothing to the story.",
    "The action sequences are chaotic and poorly choreographed, making them hard to follow.",
    "An underwhelming experience that fails to leave any lasting impression."
])

In [30]:
Old_Texts = np.array([
    "An old movie",

])

New_Texts = np.array([
    "A new movie",
])

In [31]:
transformer_model = SentenceTransformer(model_dict[Chosen_Model]["huggingface"], device="cuda") # Choose the best sentence transformer according to https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
Old_Embeddings = pd.DataFrame(transformer_model.encode(Old_Texts))
New_Embeddings = pd.DataFrame(transformer_model.encode(New_Texts))
Time_Vec= positive_to_negative_vector(Positive= New_Embeddings, Negative= Old_Embeddings)
Time_Vec

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.014276,0.013795,-0.005599,-0.025262,-0.045776,-0.039191,-0.006239,-0.033217,-0.023729,0.002896,...,0.05813,-0.013627,-0.024711,-0.021215,-0.00582,-0.033011,0.028346,0.017474,-0.021013,-0.012745


In [32]:
def project_matrix_to_vector(matrix, vector):
    """Compute the projection of a matrix onto the space spanned by the vector
    Args:
        vector: ndarray of dimension (D, 1), the vector spanning D dimensions that you want to project upon.
        matrix: ndarray of dimension (D, M), the matrix consisting of M vectors that you want to map to the subspace spanned by the vector.
    
    Returns:
        p: projection of matrix onto the subspac spanned by the columns of vector; size (D, 1)
    """
    m = matrix.to_numpy() # Turn into a matrix
    v = vector.to_numpy()[0] #Turn into a numpy array

    # Compute v dot v (denominator)
    v_dot_v = np.dot(v, v)

    # Compute projection of each row of m onto v
    projection = np.outer(np.dot(m, v) / v_dot_v, v)
    projection = pd.DataFrame(projection)

    return projection
def express_matrix_by_vector(matrix, vector):
    """Compute the projection of a matrix onto the space spanned by the vector
    Args:
        vector: ndarray of dimension (D, 1), the vector spanning D dimensions that you want to project upon.
        matrix: ndarray of dimension (D, M), the matrix consisting of M vectors that you want to map to the subspace spanned by the vector.
    
    Returns:
        projection: projection of matrix onto the subspac spanned by the columns of vector; size (D, 1)
        projection_in_1D_subspace: Each embedding projected onto 1 dimensional subspace spanned by input vector.
    """
    unit_vector = vector / np.linalg.norm(vector) # Find the unit vector for interpretatbility by dividing with its norm
    projection = project_matrix_to_vector(matrix, vector) # Find projections, so we can find lengths by finding relations in first dimension
    projection_in_1D_subspace = projection.iloc[:,0]/unit_vector.iloc[:,0][0] # Location in subspace

    return projection, projection_in_1D_subspace


In [33]:
model_dict[Chosen_Model]['Embeddings'].iloc[:,:-1]

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.000947,0.012226,0.007005,0.055534,-0.014657,0.002093,-0.074283,-0.039043,-0.032846,-0.052571,...,-0.060012,-0.017689,-0.084283,0.017695,0.004549,0.043206,-0.031605,0.011563,0.031745,0.002231
1,-0.052814,0.075769,-0.001993,0.000877,-0.011637,0.033047,-0.067636,0.008955,-0.012041,0.013748,...,-0.010560,-0.019201,0.009876,0.021684,-0.033824,-0.027393,-0.003892,0.050564,-0.012418,0.008505
2,0.009295,0.026960,-0.008366,0.004835,-0.004934,0.028715,-0.028778,-0.024467,-0.058873,0.007024,...,-0.073719,0.013301,-0.079699,0.032424,-0.007187,0.063945,-0.006870,0.012035,0.040202,-0.007758
3,0.013212,-0.013945,0.020238,-0.017202,0.064402,-0.022100,0.005692,-0.038482,-0.045942,-0.008018,...,0.012678,0.060098,-0.055245,-0.005580,0.001606,-0.046693,0.006035,0.012392,0.028939,0.009629
4,0.017525,0.003245,-0.025342,0.028730,-0.047887,0.027611,0.001204,0.022175,-0.012371,-0.023801,...,-0.042005,-0.013046,-0.054086,0.037302,0.000146,-0.001943,-0.079416,-0.028491,0.051881,-0.015033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.010382,0.053012,-0.032119,0.057019,0.025475,-0.019095,0.019149,0.024946,0.026985,0.030446,...,0.058617,0.050177,-0.005847,-0.019756,-0.009247,-0.033641,0.019529,-0.020493,-0.026690,0.000773
99996,-0.015336,-0.002935,-0.000669,-0.000911,-0.042267,0.010768,-0.054165,0.000786,-0.031114,-0.030957,...,0.030846,0.043812,-0.039896,-0.019493,-0.024774,-0.043664,-0.004756,0.036723,0.051980,0.003448
99997,-0.012683,0.065704,-0.005387,0.006981,-0.057868,0.001411,-0.013149,0.002003,-0.113928,0.008597,...,-0.000142,0.042010,-0.067297,-0.011826,-0.004276,-0.066133,0.005304,0.036001,0.012250,-0.009513
99998,-0.036789,-0.032421,0.001179,0.038486,-0.001766,0.029219,-0.119113,-0.010546,0.012335,0.051544,...,-0.010776,-0.024835,0.073117,0.004253,-0.016748,-0.011802,0.027796,0.040786,-0.010568,0.002672


In [34]:
# Save positive negative corrected embedding:
projected_variance, projection_in_1D_subspace = express_matrix_by_vector(matrix=model_dict[Chosen_Model]['Embeddings'].iloc[:,:-1], vector=Time_Vec)
### 
time_corrected_embeddings = pd.DataFrame(model_dict[Chosen_Model]['Embeddings'].iloc[:,:-1].to_numpy()-projected_variance.to_numpy())
time_corrected_embeddings['time_subspace'] = projection_in_1D_subspace
time_corrected_embeddings['rating'] = model_dict[Chosen_Model]['Embeddings']['rating']
save_corrected = f'../Data/time_corrected.csv'
time_corrected_embeddings.to_csv(save_corrected, index=False)

In [35]:
def split_for_training(dataframe):
    """Prepare dataframe for regression
    Args:
        dataframe: pandas dataframe (D, M), the matrix consisting of D-1 independent variable, for M datapoints, and the dataframe.iloc[:,-1] is the dependent variable.
    
    Returns:
        train_features: ndarray of independent training features
        train_labels: ndarray of dependent variable for training 
        eval_features: ndarray of independent evaluation features
        eval_labels: ndarray of dependent variable for training 
    """
    split = train_test_split(dataframe, train_size=0.7)
    finaleval=split[1]
    subset=split[0]

    train_features = subset.iloc[:,0:-1].to_numpy()
    train_labels = np.float32(subset.iloc[:,-1:].to_numpy())
    eval_features = finaleval.iloc[:,0:-1].to_numpy()
    eval_labels = np.float32(finaleval.iloc[:,-1:].to_numpy())

    return train_features, train_labels, eval_features, eval_labels

def linear_regression(train_features, train_labels, eval_features, eval_labels):
    """
    Perform regression and return results, summary, and model.

    Args:
        train_features: ndarray of independent training features
        train_labels: ndarray of dependent variable for training 
        eval_features: ndarray of independent evaluation features
        eval_labels: ndarray of dependent variable for training 

    Returns:
        results: DataFrame containing True and Predicted Labels
        summary: Dictionary containing R-squared, MSE, Correlation
        model: The trained regression model
    """
    import statsmodels.api as sm
    import numpy as np
    import pandas as pd

    # Regression Model
    X = sm.add_constant(train_features)  # adding a constant
    model = sm.OLS(train_labels, X).fit()

    # Predictive accuracy
    X_test = sm.add_constant(eval_features)  # add intercept to test_data
    predictions = model.predict(X_test)  # predict

    # Flatten eval_labels if necessary
    eval_labels = eval_labels.ravel()

    # Create DataFrame with true and predicted labels
    results = pd.DataFrame({
        'True Labels': eval_labels,
        'Predicted Labels': predictions
    })

    # Compute additional statistics
    mse = np.mean((results['True Labels'] - results['Predicted Labels'])**2)
    correlation = results.corr().iloc[0, 1]
    r_squared = model.rsquared

    # Return a summary as a dictionary
    summary = {
        "R-squared": r_squared,
        "MSE": mse,
        "Correlation": correlation
    }

    return results, summary, model

def pipeline(dataframe):
    # Takens in a pd dataframe with dependent variable as last collumn
    train_features, train_labels, eval_features, eval_labels= split_for_training(dataframe = dataframe)
    results, summary, model = linear_regression(train_features, train_labels, eval_features, eval_labels)
    return results, summary, model

# Create a linear regression of entire embedding space:

In [36]:
time_corrected_embeddings.drop(columns="time_subspace")

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,rating
0,-0.000681,0.011969,0.007110,0.056006,-0.013802,0.002825,-0.074166,-0.038422,-0.032403,-0.052625,...,-0.017435,-0.083821,0.018091,0.004658,0.043822,-0.032134,0.011237,0.032137,0.002469,8
1,-0.052798,0.075755,-0.001987,0.000904,-0.011588,0.033089,-0.067629,0.008990,-0.012016,0.013745,...,-0.019186,0.009903,0.021706,-0.033818,-0.027357,-0.003923,0.050545,-0.012395,0.008518,9
2,0.009014,0.027232,-0.008476,0.004338,-0.005835,0.027945,-0.028901,-0.025120,-0.059339,0.007081,...,0.013033,-0.080185,0.032007,-0.007302,0.063295,-0.006313,0.012379,0.039789,-0.008009,3
3,0.014146,-0.014848,0.020604,-0.015548,0.067399,-0.019534,0.006101,-0.036307,-0.044389,-0.008207,...,0.060990,-0.053627,-0.004191,0.001987,-0.044532,0.004179,0.011248,0.030314,0.010463,9
4,0.017105,0.003651,-0.025506,0.027986,-0.049235,0.026457,0.001021,0.021197,-0.013070,-0.023716,...,-0.013448,-0.054813,0.036677,-0.000025,-0.002915,-0.078581,-0.027976,0.051263,-0.015408,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.011085,0.052332,-0.031843,0.058264,0.027730,-0.017164,0.019456,0.026583,0.028154,0.030304,...,0.050849,-0.004629,-0.018710,-0.008960,-0.032014,0.018132,-0.021354,-0.025655,0.001401,1
99996,-0.013684,-0.004532,-0.000021,0.002012,-0.036970,0.015303,-0.053443,0.004630,-0.028368,-0.031292,...,0.045389,-0.037037,-0.017038,-0.024100,-0.039845,-0.008036,0.034701,0.054411,0.004922,10
99997,-0.009864,0.062980,-0.004281,0.011968,-0.048831,0.009148,-0.011917,0.008560,-0.109243,0.008025,...,0.044700,-0.062419,-0.007638,-0.003127,-0.059616,-0.000292,0.032552,0.016399,-0.006997,10
99998,-0.038319,-0.030942,0.000578,0.035778,-0.006674,0.025017,-0.119782,-0.014108,0.009791,0.051855,...,-0.026296,0.070467,0.001978,-0.017372,-0.015341,0.030835,0.042659,-0.012820,0.001306,10


In [37]:
# Define a dictionary with model names and corresponding data
model_data = {
    "Only subspace Model" : time_corrected_embeddings.iloc[:,-2:]
}

# Initialize a list to store summaries
summaries = []

# Iterate through the dictionary
for model_name, data in model_data.items():
    print(f"{model_name}:")
    results, summary, model = pipeline(data)  # Assuming pipeline returns (results, summary, model)
    # Add the model name to the summary dictionary and append to the list
    summary["Model Name"] = model_name
    summaries.append(summary)
    print("")

# Convert the list of summaries into a DataFrame
summary_df = pd.DataFrame(summaries)

# Display the combined summary
print("Summary of All Models:")
print(summary_df)
# Mini 0.453455     0.541113  

train_features, train_labels, eval_features, eval_labels= split_for_training(dataframe = time_corrected_embeddings.drop(columns="time_subspace"))
results, summary, model = linear_regression(train_features, train_labels, eval_features, eval_labels)
model

Only subspace Model:

Summary of All Models:
   R-squared       MSE  Correlation           Model Name
0   0.011357  8.221192     0.115816  Only subspace Model
